### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 4. Vectors: Enumerating Modules and Hecke Algebras

![Benches](images/benches.jpg)

## Setup

First, reload the algorithms from the last day ...

In [ ]:
LoadPackage("jupyterviz");
opts := rec(vertexwidth := 12, vertexheight := 12, edgecolor := "#def");;
Read("orbits.g");
Read("coxeter.g");

##  Spinning: $K$-Linear Orbit Algorithm

* In the **linear** version, the points of an orbit are linearly independent vectors in $K^n$.
* And the acting operators are like matrices in $K^{n\times n}$.

In [ ]:
spinning :=  function(aaa, x, under)
    local list,  a,  y,  z;
    list := [x];
    for y in list do
        for a in aaa do
            z := under(y, a);
            if not z in VectorSpace(Rationals, list) then
                Add(list, z);
            fi;
        od;
    od;
    return list;
end;

### Example

In [ ]:
A := [
[0,1,0,0,0],
[0,0,1,0,0],
[0,0,0,1,0],
[0,0,0,0,1],
-[1,1,1,1,1],
];

In [ ]:
spinning([A], [0,0,1,0,0], OnRight);

In [ ]:
spinning([A], [1,1,0,1,1], OnRight);

In [ ]:
spinning([A], [1,0,1,0,1], OnRight);

* Example $G(3,3,3)$.

##  Spinning with Images

* We will need to be able to convert between sparse and dense representations of vectors.

In [ ]:
sparseVec:= function(vec)
    local   poss;
    poss:= PositionsProperty(vec, v -> v <> Zero(v));
    return rec(pos := poss,  val:= vec{poss});
end;

denseVec:= function(l, v)
    local   vec;
    vec := 0*[1..l];  vec{v.pos} := v.val;
    return vec;
end;

In [ ]:
vec := [0,0,1,1,0];
v := sparseVec(vec);

In [ ]:
denseVec(Length(vec), v) = vec;

* spinning with images (cf. orbit with images)

* In the linear version the images are vectors: linear combination of the base points.

In [ ]:
spinning_with_images :=  function(aaa, x, under)
    local   list,  images,  i,  y,  k,  a,  z,  v;
    list := [x];  images := List(aaa, x-> []);  i := 0;
    while i < Length(list) do
        i := i+1;  y := list[i];
        for k in [1..Length(aaa)] do
            a := aaa[k];  z := under(y, a);
            v := SolutionMat(list, z);
            if v = fail then
                Add(list, z);
                v := rec(pos := [Length(list)], val := [1]);
            else
                v := sparseVec(v);
            fi;
            images[k][i]:= v;
        od;
    od;
    return rec(list := list, images := images);
end;

### Example: Specht-Module

* Everybody knows that Specht modules are made of Standard Young Tableaus ...
* Here is a random one, of shape $\lambda = (2,2,1)$ (for the rest see the exercise on SYTs in part 1).

In [ ]:
tab := [
  [1,4],
  [2,5],
  [3],
];

* and the corresponding symmetric group

In [ ]:
N := Sum(tab, Length);;  gens := transpositions(N);

* each tableau (standard or not) yields a (row) word, recording for each number the row it is in.

In [ ]:
wordTab := function(tab)
    local   word,  i;
    word := [];
    for i in [1..Length(tab)] do
        word{tab[i]} := 0*tab[i] + i;
    od;
    return word;
end;

In [ ]:
wordTab(tab);

* the column stabilizer is the stabilizer of the column word.
* the orbit of a tableau under the column stabilizer is a (column) tabloid.

In [ ]:
colWord := wordTab(TransposedMat(tab));
stab := takeAway(orbit_with_stabilizer(gens, colWord, Permuted).stab, ());
tabs := orbit_with_transversal(stab, tab, OnTuplesTuples);

* the symmetric group acts on the row words

In [ ]:
orb := orbit_with_images(gens, wordTab(tab), Permuted);
words := orb.list;;
perms := List(orb.images, PermList);

* the polytabloid is a signed combination of tabloids, written as a vector in the permutation module.

In [ ]:
vec := List(words, x-> 0);
for i in [1..Length(tabs.list)] do
    vec[Position(words, wordTab(tabs.list[i]))] := SignPerm(tabs.reps[i]);
od;

* apply spinning to our polytabloid

In [ ]:
vvv:= spinning_with_images(perms, vec, Permuted);

* how to convert a list of sparse vectors into a matrix (cf. `PermList`)

In [ ]:
mat_list:= list -> List(list, v -> denseVec(Length(list), v));

In [ ]:
mats := List(vvv.images, mat_list);

In [ ]:
for m in mats do PrintArray(m);  Print("\n"); od;

## Schreier Matrices and the Group Algebra

* Another way to produce matrices from an orbit is to read the resulting permutations as **permutation matrices**.
* Or, to use the Schreier generators as entries, rather than just $1$.

In [ ]:
gens := transpositions(4);;
orb := orbit_with_images(gens, 4, OnPoints);;
mats := List(orb.images, x -> PermutationMat(PermList(x), Length(x)));

In [ ]:
for m in mats do PrintArray(m);  Print("\n"); od;

## Exercises, etc.

* Expand `spinning` into `spinning_with_matrices` ...
* Compute representing matrices for the generators acting on the Specht module.